In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
import random

num_of_centroids=4096
num_train_samples=200
num_test_samples=40
K = 11
all_image_idx=random.sample(range(0,1000),num_train_samples+num_test_samples)

In [ ]:
from PIL import Image
from skimage.filters import unsharp_mask

def remove_noise (image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    X = cv2.bilateralFilter(gray, 15, sigmaColor=10, sigmaSpace=10)
    median = cv2.medianBlur(X, 5)
    result_2 = unsharp_mask(median, radius=10, amount=4)*255
    result_2 = np.uint8(result_2)
    sharpen = cv2.Canny(result_2, 100,250)

    # Create a kernel for morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    # # Close the edges using morphological closing
    closed_edges = cv2.morphologyEx(sharpen, cv2.MORPH_CLOSE, kernel)
    return closed_edges

def show_images(images, titles=None):
    n_ims = len(images)
    if titles is None:
        titles = ['(%d)' % i for i in range(1, n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image, title in zip(images, titles):
        a = fig.add_subplot(1, n_ims, n)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()

In [ ]:
all_des=np.empty((1,128))

sift = cv2.SIFT_create()

num_of_desc=[]

for i in range (num_train_samples):
    image = cv2.imread("../../fonts-dataset/IBM Plex Sans Arabic/"+str(all_image_idx[i])+".jpeg")
    removed_noise = remove_noise(image)
    kp , descriptors= sift.detectAndCompute(removed_noise,None)
    if descriptors is not None:
        all_des=np.vstack((all_des,descriptors))
    print("processing type 1 image"+str(i))

num_of_desc+=[all_des.shape[0]]
print("FINISHED READING FIRST SET OF IMAGES")

for i in range (num_train_samples):
    image = cv2.imread("../../fonts-dataset/Lemonada/"+str(all_image_idx[i])+".jpeg")
    removed_noise = remove_noise(image)
    kp , descriptors= sift.detectAndCompute(removed_noise,None)
    if descriptors is not None:
        all_des=np.vstack((all_des,descriptors))
    print("processing type 2 image"+str(i))

num_of_desc+=[all_des.shape[0]]
print("FINISHED READING SECOND SET OF IMAGES")

for i in range (num_train_samples):
    image = cv2.imread("../../fonts-dataset/Marhey/"+str(all_image_idx[i])+".jpeg")
    removed_noise = remove_noise(image)
    kp , descriptors= sift.detectAndCompute(removed_noise,None)
    if descriptors is not None:
        all_des=np.vstack((all_des,descriptors))
    print("processing type 3 image"+str(i))

num_of_desc+=[all_des.shape[0]]
print("FINISHED READING THIRD SET OF IMAGES")


for i in range (num_train_samples):
    image = cv2.imread("../../fonts-dataset/Scheherazade New/"+str(all_image_idx[i])+".jpeg")
    removed_noise = remove_noise(image)
    kp , descriptors= sift.detectAndCompute(removed_noise,None)
    if descriptors is not None:
        all_des=np.vstack((all_des,descriptors))
    print("processing type 4 image"+str(i))

num_of_desc+=[all_des.shape[0]]
print("FINISHED READING FOURTH SET OF IMAGES")

desc_labels=np.zeros(all_des.shape[0])
desc_labels[num_of_desc[0]:num_of_desc[1]]=1
desc_labels[num_of_desc[1]:num_of_desc[2]]=2
desc_labels[num_of_desc[2]:num_of_desc[3]]=3

In [ ]:
# different options for kmeans
# - mini batches
# - limiting the number of descriptors
kmeans=MiniBatchKMeans(n_clusters=num_of_centroids,batch_size=num_of_centroids*1024,max_iter=20).fit(X=all_des)

In [ ]:
# for each centroid, calculate how it is near to each label
centroids_labels=np.zeros((num_of_centroids,4))
for i in range(num_of_centroids):
    centroids_labels[i][0]=(np.sum(desc_labels[kmeans.labels_==i]==0))  ## kmeans.labels_ -> array to map each descriptor to a centroid
    centroids_labels[i][1]=(np.sum(desc_labels[kmeans.labels_==i]==1))
    centroids_labels[i][2]=(np.sum(desc_labels[kmeans.labels_==i]==2))
    centroids_labels[i][3]=(np.sum(desc_labels[kmeans.labels_==i]==3))
    centroids_labels[i]/=np.sum(centroids_labels[i])
print(centroids_labels[0][0], centroids_labels[0][1], centroids_labels[0][2], centroids_labels[0][3])

In [ ]:
def predict(path,centroids,centroids_labels):
    image=cv2.imread(path)
    removed_noise = remove_noise(image)
    _ , descriptors= sift.detectAndCompute(removed_noise,None)

    predections=[0.0,0.0,0.0,0.0]
    if descriptors is not None:
        for des in descriptors:
            idx=kmeans.predict([des])
            dist=np.linalg.norm(des-centroids[idx])
            if dist == 0:
                dist = 0.0000001
            predections+=(centroids_labels[idx]/dist)
        return np.argmax(predections)
    else:
        return -1

num_right0=0
num_right1=0
num_right2=0
num_right3=0

predictions=[]

for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])
    path="../../fonts-dataset/IBM Plex Sans Arabic/"+rand_idx+".jpeg"

    predicted=predict(path,kmeans.cluster_centers_,centroids_labels)
    print("image",(rand_idx), "type0: ",predicted)
    num_right0+=predicted==0
    predictions.append(predicted)

for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])
    path="../../fonts-dataset/Lemonada/"+rand_idx+".jpeg"

    predicted=predict(path,kmeans.cluster_centers_,centroids_labels)
    print("image",(rand_idx), "type1: ",predicted)
    num_right1+=predicted==1
    predictions.append(predicted)

for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])

    path="../../fonts-dataset/Marhey/"+rand_idx+".jpeg"

    predicted=predict(path,kmeans.cluster_centers_,centroids_labels)
    print("image",(rand_idx), "type2: ",predicted)
    num_right2+=predicted==2
    predictions.append(predicted)

for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])

    path="../../fonts-dataset/Scheherazade New/"+rand_idx+".jpeg"

    predicted=predict(path,kmeans.cluster_centers_,centroids_labels)
    print("image",(rand_idx), "type3: ",predicted)
    num_right3+=predicted==3
    predictions.append(predicted)

In [ ]:
print(num_right0)
print(num_right1)
print(num_right2)
print(num_right3)

print((num_right0+num_right1+num_right2+num_right3)/(4*num_test_samples))

In [ ]:
from sklearn.neighbors import NearestNeighbors

def predict_knn(path,centroids_labels,knn):
    image=cv2.imread(path)
    removed_noise = remove_noise(image)
    _ , descriptors= sift.detectAndCompute(removed_noise,None)

    predections=[0.0,0.0,0.0,0.0]
    if descriptors is not None:
        for des in descriptors:
            neigh=knn.kneighbors([des])
            for i in range(K):
                predections+=(centroids_labels[neigh[1][0][i]]/neigh[0][0][i])
        print(predections)
        return np.argmax(predections)
    else:
        return -1

knn = NearestNeighbors(n_neighbors=K)
classes=np.arange(0, num_of_centroids, 1, dtype=int)
knn.fit(kmeans.cluster_centers_,classes)

predictions=[]

for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])
    path="../../fonts-dataset/IBM Plex Sans Arabic/"+rand_idx+".jpeg"

    predicted=predict_knn(path,centroids_labels,knn)
    print("image",(rand_idx), "type0: ",predicted)
    num_right0+=predicted==0
    predictions+=predicted

for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])
    path="../../fonts-dataset/Lemonada/"+rand_idx+".jpeg"

    predicted=predict_knn(path,centroids_labels,knn)
    print("image",(rand_idx), "type1: ",predicted)
    num_right1+=predicted==1
    predictions+=predicted

for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])

    path="../../fonts-dataset/Marhey/"+rand_idx+".jpeg"

    predicted=predict_knn(path,centroids_labels,knn)
    print("image",(rand_idx), "type2: ",predicted)
    num_right2+=predicted==2
    predictions+=predicted

for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])

    path="../../fonts-dataset/Scheherazade New/"+rand_idx+".jpeg"

    predicted=predict_knn(path,centroids_labels,knn)
    print("image",(rand_idx), "type3: ",predicted)
    num_right3+=predicted==3
    predictions+=predicted

In [ ]:
print(num_right0)
print(num_right1)
print(num_right2)
print(num_right3)

print((num_right0+num_right1+num_right2+num_right3)/(4*num_test_samples))

In [ ]:
# SVM on ther original des

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

all_des=[]
labels=[]
sift = cv2.SIFT_create(500)

for i in range (10):
    image = cv2.imread("../../fonts-dataset/Lemonada/"+str(i)+".jpeg")
    removed_noise = remove_noise(image)
    kp , descriptors= sift.detectAndCompute(removed_noise,None)
    
    if descriptors is not None:
        for des in descriptors:
            all_des.append(des)
        # append number of labels to the labels array equal to the number of descriptors
        for j in range (descriptors.shape[0]):
            labels=np.append(labels,0)        
    


    image2 = cv2.imread("../../fonts-dataset/Marhey/"+str(i)+".jpeg")
    removed_noise2 = remove_noise(image2)
    kp , descriptors= sift.detectAndCompute(removed_noise2,None)

    if descriptors is not None:
        for des in descriptors:
            all_des.append(des)  
        # append number of labels to the labels array equal to the number of descriptors
        for j in range (descriptors.shape[0]):
            labels=np.append(labels,1)

    image3 = cv2.imread("../../fonts-dataset/IBM Plex Sans Arabic/"+str(i)+".jpeg")
    removed_noise3 = remove_noise(image3)
    kp , descriptors= sift.detectAndCompute(removed_noise3,None)

    if descriptors is not None:
        for des in descriptors:
            all_des.append(des)
        # append number of labels to the labels array equal to the number of descriptors
        for j in range (descriptors.shape[0]):
            labels=np.append(labels,2)

    image4 = cv2.imread("../../fonts-dataset/Scheherazade New/"+str(i)+".jpeg")
    removed_noise4 = remove_noise(image4)
    kp , descriptors= sift.detectAndCompute(removed_noise4,None)

    if descriptors is not None:
        for des in descriptors:
            all_des.append(des)
        # append number of labels to the labels array equal to the number of descriptors
        for j in range (descriptors.shape[0]):
            labels=np.append(labels,3)
    
    print(i)

all_des = np.array(all_des)
print(all_des.shape)

X_train, X_test, y_train, y_test = train_test_split(all_des, labels, test_size=0.2, random_state=42)
svm = SVC(kernel='poly', C=0.1, random_state=0, coef0=1, degree=4, gamma=10.0,class_weight= None)
print("fares1")
svm.fit(X_train, y_train)
print("fares1")
y_pred = svm.predict(X_test)
print(accuracy_score(y_test, y_pred)*100)

In [ ]:
# Fixing individual predictions
def remove_noise22 (image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    X = cv2.bilateralFilter(gray, 15, sigmaColor=10, sigmaSpace=10)
    median = cv2.medianBlur(X, 5)
    result_2 = unsharp_mask(median, radius=10, amount=4)*255
    result_2 = np.uint8(result_2)
    sharpen = cv2.Canny(result_2, 100,250)
    
    # Create a kernel for morphological operations
    # kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    # 
    # # Close the edges using morphological closing
    # closed_edges = cv2.morphologyEx(sharpen, cv2.MORPH_CLOSE, kernel)
    return sharpen

def predict22(path,centroids,centroids_labels):
    image=cv2.imread(path)
    removed_noise = remove_noise22(image)
    _ , descriptors= sift.detectAndCompute(removed_noise,None)

    predections=[0.0,0.0,0.0,0.0]
    if descriptors is not None:
        for des in descriptors:
            idx=kmeans.predict([des])
            dist=np.linalg.norm(des-centroids[idx])
            if dist == 0:
                dist = 0.0000001
            predections+=(centroids_labels[idx]/dist)
        print(predections)
        return np.argmax(predections)
    else:
        return -1
path="../../fonts-dataset/Scheherazade New/840.jpeg"

prediction=predict22(path,kmeans.cluster_centers_,centroids_labels)
print("image",(599), "type3: ",prediction)


In [ ]:
print("image",(599), "type3: ",prediction)

In [ ]:
image = cv2.imread("../../fonts-dataset/Scheherazade New/599.jpeg")
removed_noise = remove_noise22(image)
# save the image
cv2.imwrite("1.jpeg", removed_noise)

grayy = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
sharpen = cv2.Canny(grayy, 100,250)
cv2.imwrite("2.jpeg", sharpen)

result_2 = unsharp_mask(grayy, radius=10, amount=4)*255
result_2 = np.uint8(result_2)
#X = cv2.bilateralFilter(result_2, 15, sigmaColor=10, sigmaSpace=10)

# kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
# 
# # Close the edges using morphological closing
# closed_edges = cv2.morphologyEx(grayy, cv2.MORPH_DILATE, kernel)

# median = cv2.medianBlur(X, 5)
cv2.imwrite("3.jpeg", result_2)